In [1]:
import sys, os
sys.path.insert(0, '../../../residual2vec_')

In [2]:
os.environ['DISABLE_WANDB'] = 'true'

#disable epoch level tqdm
os.environ['DISABLE_TQDM'] = 'true'

In [3]:
import numpy as np
import pandas as pd
import pickle as pkl
from scipy import sparse

In [4]:
from utils import snakemake_utils, score

/data/sg/ashutiwa/miniconda3/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using GPU: cuda:0


In [5]:
DIR = '../../../final_crosswalk/airport/airport_one/'
adj = snakemake_utils.get_adj_mat_from_path( DIR + "airport_adj.npz")
test_adj = snakemake_utils.get_adj_mat_from_path(DIR + "airport_test_adj.npz")
embs = np.load(DIR + 'airport_fairwalk_deepwalk.npy')
y = snakemake_utils.get_dataset("airport").get_grouped_col()

In [6]:
adj.shape, test_adj.shape, embs.shape

((2898, 2898), (2898, 2898), (2898, 128))

In [7]:
batch_size = 2000
edges = snakemake_utils.return_new_graph(embs, 5)

In [8]:


n_nodes = len(y) # number of nodes
n_edges = edges.shape[0] # number of nodes
uy, y = np.unique(y, return_inverse=True) # to ensure that the labels are continuous integers starting from zero
K = len(uy) # number of classes

# We need two groups at least
assert K >= 2

# Group membership matrix, where U[i,k] = 1 if node $i$ belongs to group $k$
U = sparse.csr_matrix((np.ones_like(y), (np.arange(n_nodes), y)), shape=(n_nodes, K))

# Number of nodes in each group
Nk = np.array(U.sum(axis = 0)).reshape(-1)

# Adjacency matrix
A = sparse.csr_matrix((np.ones(n_edges), (edges["source"].values, edges["target"].values)), shape=(n_nodes, n_nodes))

# Make sure that the adjacency matrix is symemtric
A = A + A.T
A.data = A.data *0 + 1

# Calculate the number of edges that appear in each group
M = U.T @ A @ U

# Calculate the edge density
Mdenom = np.outer(Nk, Nk) - np.diag(Nk)
P = M / Mdenom
# Calculate the statistical parity

probs = np.array(P[np.triu_indices(K)]).reshape(-1)


In [9]:
P[np.triu_indices(K)]

matrix([[1.95082777e-02, 6.25390869e-05, 5.06119668e-04, 7.56200847e-04,
         1.56896306e-04, 6.14820883e-03, 4.81403512e-05, 1.97321602e-04,
         6.48271023e-05, 8.78602250e-03, 5.04567411e-04, 3.55197727e-04,
         1.12262522e-02, 0.00000000e+00, 2.53857641e-02]])

In [10]:
M

<5x5 sparse matrix of type '<class 'numpy.float64'>'
	with 23 stored elements in Compressed Sparse Column format>

In [11]:
Nk

array([ 285, 1066,  721,  580,  246])

In [14]:
np.outer(Nk, Nk)

(5, 5)